In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import time

from model import Generator, Discriminator
from config import irgan_config
from data_utils import RecDataset, DataProvider
from evaluation.rec_evaluator import RecEvaluator

In [2]:
epochs = irgan_config.epochs
batch_size = irgan_config.batch_size
epochs_d = 15
epochs_g = 50
emb_dim = irgan_config.emb_dim
eta_G = irgan_config.eta_G
eta_D = irgan_config.eta_D
device = irgan_config.device
weight_decay_g = irgan_config.weight_decay_g
weight_decay_d = irgan_config.weight_decay_d

In [3]:
# Hyper-parameters and datset-specific parameters
rec_dataset = RecDataset(irgan_config.dir_path)
all_users = rec_dataset.get_users()
all_items = rec_dataset.get_items()
num_users = rec_dataset.get_num_users()
num_items = rec_dataset.get_num_items()
bought_mask = rec_dataset.get_bought_mask().to(device)
eval_dict = rec_dataset.get_interaction_records("test")
train_ui = rec_dataset.get_user_item_pairs()

In [4]:
dp = DataProvider(device)
evaluator = RecEvaluator(eval_dict, None, device)

In [5]:
writer_dis = SummaryWriter("runs/IRGAN-Adam-dns-gen-dis/Discriminator")
writer_gen = SummaryWriter("runs/IRGAN-Adam-dns-gen-dis/Generator")

In [6]:
G = Generator(num_users, num_items, emb_dim, bought_mask)
D = Discriminator(num_users, num_items, emb_dim)
G = G.to(device)
D = D.to(device)

loss_D = nn.BCELoss()
optimizer_G = torch.optim.SGD(G.parameters(), lr = eta_G)
optimizer_D = torch.optim.SGD(D.parameters(), lr = eta_D, weight_decay = weight_decay_d)

In [7]:
pretrained_model = torch.load("./pretrained_models/pretrained_model_dns.pkl",map_location=irgan_config.device)
G.load_state_dict(pretrained_model)
D.load_state_dict(pretrained_model)

<All keys matched successfully>

In [8]:
for epoch in range(epochs):
    print(
            "[Epoch %d/%d]"
            % (epoch+1, epochs)
        )  
    time_epoch_start = time.time()
    
    # ---------------------
    #  Train Discriminator
    # ---------------------   
    D.train()
    G.eval()
    for epoch_d in range(epochs_d):
        time_d_start = time.time()
        if(epoch_d % 5 == 0):
            train_set_d = dp.prepare_data_for_discriminator(G, train_ui, batch_size = batch_size)
        loss_d_epoch = 0
        for users, items, labels in train_set_d:
            dis_score = D(users,items)
            loss_d = loss_D(dis_score, labels)
            loss_d_epoch += loss_d.item()
            optimizer_D.zero_grad()
            loss_d.backward()
            optimizer_D.step()
            
        time_d_end = time.time()
        loss_d_epoch /= len(train_set_d)
        print(
            "\t[Discriminator][Epochs %d/%d] [D epoch loss: %6.5f] [Time:%6.5f] "
            % (epoch_d+1, epochs_d, loss_d_epoch, time_d_end - time_d_start)
        )
        writer_dis.add_scalar('Loss/Discriminator', loss_d_epoch, epoch*epochs_d + epoch_d)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(D, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_dis.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MAP', map10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MRR', mrr10, epoch*epochs_d + epoch_d)
        
    # --------------------- 
    #  Train Generator
    #
    # For generator
    #   Generate K user-item pairs
    #   Leveraging Policy Gradient to update parameters of generator
    #
    # --------------------- 
    D.eval()
    G.train()
    for epoch_g in range(epochs_g):
        time_g_start = time.time()
        train_set_g = dp.prepare_data_for_generator(all_users, batch_size)
        loss_g_epoch = 0
        for fake_users, in train_set_g:
            fake_items, fake_probs, fake_p_n = G.sample_items_for_users(fake_users, k = 256,\
                                                                        temperature=1, lambda_bought=0.2)
            fake_users = fake_users.view(-1,1).expand_as(fake_items).contiguous()
            fake_users = fake_users.view(-1)
            fake_items = fake_items.view(-1)
            fake_probs = fake_probs.view(-1)
            fake_p_n = fake_p_n.view(-1)
            
            log_fake_probs = torch.log(fake_probs.clamp(1e-8))
            fake_probs = fake_probs.detach()
            fake_p_n = fake_p_n.detach()
            
            reward = (2 * D(fake_users, fake_items) - 1).detach()*(fake_probs/fake_p_n)
            loss_g = -torch.mean(log_fake_probs*reward)
            loss_g_epoch += loss_g
            
            optimizer_G.zero_grad()  
            loss_g.backward() 
            optimizer_G.step()
        time_g_end = time.time()
        print(
            "\t[Generator][Epochs %d/%d] [G epoch loss: %6.5f] [Time:%6.5f]"
            % (epoch_g + 1, epochs_g, loss_g_epoch, time_g_end - time_g_start )
        ) 
        writer_gen.add_scalar('Loss/Generator', loss_g_epoch, epoch*epochs_g + epoch_g)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(G, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_gen.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MAP', map10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MRR', mrr10, epoch*epochs_g + epoch_g)
    time_epoch_end = time.time()
writer_dis.close()    
writer_gen.close()

[Epoch 1/200]
	[Discriminator][Epochs 1/15] [D epoch loss: 3.62698] [Time:4.93931] 
	[Discriminator][Epochs 2/15] [D epoch loss: 3.35926] [Time:6.52765] 
	[Discriminator][Epochs 3/15] [D epoch loss: 3.10019] [Time:8.24987] 
	[Discriminator][Epochs 4/15] [D epoch loss: 2.85313] [Time:8.35942] 
	[Discriminator][Epochs 5/15] [D epoch loss: 2.62212] [Time:8.51841] 
	[Discriminator][Epochs 6/15] [D epoch loss: 2.41095] [Time:8.54261] 
	[Discriminator][Epochs 7/15] [D epoch loss: 2.22235] [Time:7.84082] 
	[Discriminator][Epochs 8/15] [D epoch loss: 2.05639] [Time:8.06824] 
	[Discriminator][Epochs 9/15] [D epoch loss: 1.91149] [Time:8.15625] 
	[Discriminator][Epochs 10/15] [D epoch loss: 1.78514] [Time:8.40756] 
	[Discriminator][Epochs 11/15] [D epoch loss: 1.67515] [Time:10.53761] 
	[Discriminator][Epochs 12/15] [D epoch loss: 1.57838] [Time:10.52518] 
	[Discriminator][Epochs 13/15] [D epoch loss: 1.49326] [Time:9.85580] 
	[Discriminator][Epochs 14/15] [D epoch loss: 1.41821] [Time:10.34052]

	[Generator][Epochs 43/50] [G epoch loss: 1.53125] [Time:0.04370]
	[Generator][Epochs 44/50] [G epoch loss: 1.45683] [Time:0.03710]
	[Generator][Epochs 45/50] [G epoch loss: 1.12200] [Time:0.03499]
	[Generator][Epochs 46/50] [G epoch loss: 1.05422] [Time:0.03451]
	[Generator][Epochs 47/50] [G epoch loss: 1.14983] [Time:0.03520]
	[Generator][Epochs 48/50] [G epoch loss: 0.87894] [Time:0.03725]
	[Generator][Epochs 49/50] [G epoch loss: 1.17012] [Time:0.04385]
	[Generator][Epochs 50/50] [G epoch loss: 1.44319] [Time:0.03530]
[Epoch 3/200]
	[Discriminator][Epochs 1/15] [D epoch loss: 1.16847] [Time:8.90055] 
	[Discriminator][Epochs 2/15] [D epoch loss: 1.13607] [Time:9.09385] 
	[Discriminator][Epochs 3/15] [D epoch loss: 1.10590] [Time:8.79506] 
	[Discriminator][Epochs 4/15] [D epoch loss: 1.07771] [Time:9.83827] 
	[Discriminator][Epochs 5/15] [D epoch loss: 1.05129] [Time:9.86543] 
	[Discriminator][Epochs 6/15] [D epoch loss: 1.02645] [Time:10.35263] 
	[Discriminator][Epochs 7/15] [D epoc

	[Generator][Epochs 34/50] [G epoch loss: -5.25803] [Time:0.03757]
	[Generator][Epochs 35/50] [G epoch loss: -4.96122] [Time:0.03770]
	[Generator][Epochs 36/50] [G epoch loss: -4.81404] [Time:0.03768]
	[Generator][Epochs 37/50] [G epoch loss: -4.96389] [Time:0.03783]
	[Generator][Epochs 38/50] [G epoch loss: -4.86822] [Time:0.03856]
	[Generator][Epochs 39/50] [G epoch loss: -4.94245] [Time:0.02904]
	[Generator][Epochs 40/50] [G epoch loss: -4.76911] [Time:0.03679]
	[Generator][Epochs 41/50] [G epoch loss: -4.70288] [Time:0.03715]
	[Generator][Epochs 42/50] [G epoch loss: -4.64342] [Time:0.03769]
	[Generator][Epochs 43/50] [G epoch loss: -4.71521] [Time:0.03614]
	[Generator][Epochs 44/50] [G epoch loss: -5.05813] [Time:0.03738]
	[Generator][Epochs 45/50] [G epoch loss: -4.99908] [Time:0.02970]
	[Generator][Epochs 46/50] [G epoch loss: -4.86610] [Time:0.03546]
	[Generator][Epochs 47/50] [G epoch loss: -4.72121] [Time:0.03639]
	[Generator][Epochs 48/50] [G epoch loss: -4.85132] [Time:0.03

	[Generator][Epochs 25/50] [G epoch loss: -10.23397] [Time:0.03700]
	[Generator][Epochs 26/50] [G epoch loss: -10.05613] [Time:0.05649]
	[Generator][Epochs 27/50] [G epoch loss: -10.07112] [Time:0.04205]
	[Generator][Epochs 28/50] [G epoch loss: -10.11315] [Time:0.04473]
	[Generator][Epochs 29/50] [G epoch loss: -10.07860] [Time:0.04513]
	[Generator][Epochs 30/50] [G epoch loss: -10.08309] [Time:0.03904]
	[Generator][Epochs 31/50] [G epoch loss: -10.07588] [Time:0.04598]
	[Generator][Epochs 32/50] [G epoch loss: -10.14597] [Time:0.03963]
	[Generator][Epochs 33/50] [G epoch loss: -10.00496] [Time:0.03658]
	[Generator][Epochs 34/50] [G epoch loss: -9.73969] [Time:0.02902]
	[Generator][Epochs 35/50] [G epoch loss: -9.66742] [Time:0.02890]
	[Generator][Epochs 36/50] [G epoch loss: -9.64904] [Time:0.03077]
	[Generator][Epochs 37/50] [G epoch loss: -9.84753] [Time:0.03323]
	[Generator][Epochs 38/50] [G epoch loss: -9.86449] [Time:0.03326]
	[Generator][Epochs 39/50] [G epoch loss: -10.01141] 

	[Generator][Epochs 15/50] [G epoch loss: -14.29798] [Time:0.03536]
	[Generator][Epochs 16/50] [G epoch loss: -14.37839] [Time:0.03552]
	[Generator][Epochs 17/50] [G epoch loss: -14.29481] [Time:0.04397]
	[Generator][Epochs 18/50] [G epoch loss: -14.39123] [Time:0.03187]
	[Generator][Epochs 19/50] [G epoch loss: -14.21812] [Time:0.03794]
	[Generator][Epochs 20/50] [G epoch loss: -14.18412] [Time:0.03583]
	[Generator][Epochs 21/50] [G epoch loss: -13.91389] [Time:0.03678]
	[Generator][Epochs 22/50] [G epoch loss: -14.01988] [Time:0.03910]
	[Generator][Epochs 23/50] [G epoch loss: -14.20232] [Time:0.03961]
	[Generator][Epochs 24/50] [G epoch loss: -14.21714] [Time:0.03709]
	[Generator][Epochs 25/50] [G epoch loss: -14.07021] [Time:0.03567]
	[Generator][Epochs 26/50] [G epoch loss: -14.04667] [Time:0.02990]
	[Generator][Epochs 27/50] [G epoch loss: -14.16492] [Time:0.04455]
	[Generator][Epochs 28/50] [G epoch loss: -13.95156] [Time:0.04453]
	[Generator][Epochs 29/50] [G epoch loss: -14.01

...

	[Discriminator][Epochs 14/15] [D epoch loss: 0.45153] [Time:9.31753] 
	[Discriminator][Epochs 15/15] [D epoch loss: 0.45141] [Time:10.18149] 
	[Generator][Epochs 1/50] [G epoch loss: -31.26971] [Time:0.03007]
	[Generator][Epochs 2/50] [G epoch loss: -31.21078] [Time:0.03976]
	[Generator][Epochs 3/50] [G epoch loss: -31.38544] [Time:0.03809]
	[Generator][Epochs 4/50] [G epoch loss: -31.37264] [Time:0.03651]
	[Generator][Epochs 5/50] [G epoch loss: -31.39149] [Time:0.03917]
	[Generator][Epochs 6/50] [G epoch loss: -31.48347] [Time:0.03771]
	[Generator][Epochs 7/50] [G epoch loss: -31.46687] [Time:0.04142]
	[Generator][Epochs 8/50] [G epoch loss: -31.44173] [Time:0.03918]
	[Generator][Epochs 9/50] [G epoch loss: -31.21495] [Time:0.03692]
	[Generator][Epochs 10/50] [G epoch loss: -31.14386] [Time:0.03693]
	[Generator][Epochs 11/50] [G epoch loss: -31.10086] [Time:0.03718]
	[Generator][Epochs 12/50] [G epoch loss: -31.20457] [Time:0.03472]
	[Generator][Epochs 13/50] [G epoch loss: -31.1402

	[Discriminator][Epochs 4/15] [D epoch loss: 0.45303] [Time:8.52940] 
	[Discriminator][Epochs 5/15] [D epoch loss: 0.45289] [Time:10.02285] 
	[Discriminator][Epochs 6/15] [D epoch loss: 0.45280] [Time:9.77077] 
	[Discriminator][Epochs 7/15] [D epoch loss: 0.45267] [Time:10.13073] 
	[Discriminator][Epochs 8/15] [D epoch loss: 0.45254] [Time:10.05353] 
	[Discriminator][Epochs 9/15] [D epoch loss: 0.45241] [Time:9.27097] 
	[Discriminator][Epochs 10/15] [D epoch loss: 0.45229] [Time:10.35726] 
	[Discriminator][Epochs 11/15] [D epoch loss: 0.45217] [Time:10.00787] 
	[Discriminator][Epochs 12/15] [D epoch loss: 0.45204] [Time:10.24808] 
	[Discriminator][Epochs 13/15] [D epoch loss: 0.45192] [Time:8.85717] 
	[Discriminator][Epochs 14/15] [D epoch loss: 0.45179] [Time:8.94747] 
	[Discriminator][Epochs 15/15] [D epoch loss: 0.45167] [Time:10.09193] 
	[Generator][Epochs 1/50] [G epoch loss: -31.28365] [Time:0.04066]
	[Generator][Epochs 2/50] [G epoch loss: -31.13978] [Time:0.03781]
	[Generator][

	[Generator][Epochs 44/50] [G epoch loss: -31.08413] [Time:0.03589]
	[Generator][Epochs 45/50] [G epoch loss: -30.98494] [Time:0.03941]
	[Generator][Epochs 46/50] [G epoch loss: -31.06293] [Time:0.03792]
	[Generator][Epochs 47/50] [G epoch loss: -31.15744] [Time:0.03702]
	[Generator][Epochs 48/50] [G epoch loss: -31.14371] [Time:0.03790]
	[Generator][Epochs 49/50] [G epoch loss: -31.15547] [Time:0.03814]
	[Generator][Epochs 50/50] [G epoch loss: -31.26551] [Time:0.03765]
[Epoch 196/200]
	[Discriminator][Epochs 1/15] [D epoch loss: 0.45384] [Time:10.13594] 
	[Discriminator][Epochs 2/15] [D epoch loss: 0.45371] [Time:9.99479] 
	[Discriminator][Epochs 3/15] [D epoch loss: 0.45359] [Time:10.37801] 
	[Discriminator][Epochs 4/15] [D epoch loss: 0.45346] [Time:9.41432] 
	[Discriminator][Epochs 5/15] [D epoch loss: 0.45333] [Time:8.81005] 
	[Discriminator][Epochs 6/15] [D epoch loss: 0.45322] [Time:9.77309] 
	[Discriminator][Epochs 7/15] [D epoch loss: 0.45310] [Time:9.88174] 
	[Discriminator]

	[Generator][Epochs 34/50] [G epoch loss: -30.83014] [Time:0.03708]
	[Generator][Epochs 35/50] [G epoch loss: -30.90751] [Time:0.03675]
	[Generator][Epochs 36/50] [G epoch loss: -30.93801] [Time:0.03805]
	[Generator][Epochs 37/50] [G epoch loss: -31.01251] [Time:0.03744]
	[Generator][Epochs 38/50] [G epoch loss: -31.07014] [Time:0.03717]
	[Generator][Epochs 39/50] [G epoch loss: -31.08407] [Time:0.03682]
	[Generator][Epochs 40/50] [G epoch loss: -31.10408] [Time:0.03787]
	[Generator][Epochs 41/50] [G epoch loss: -31.07753] [Time:0.03091]
	[Generator][Epochs 42/50] [G epoch loss: -31.07934] [Time:0.03950]
	[Generator][Epochs 43/50] [G epoch loss: -31.11245] [Time:0.03726]
	[Generator][Epochs 44/50] [G epoch loss: -31.08475] [Time:0.03629]
	[Generator][Epochs 45/50] [G epoch loss: -31.13664] [Time:0.03607]
	[Generator][Epochs 46/50] [G epoch loss: -31.01450] [Time:0.03813]
	[Generator][Epochs 47/50] [G epoch loss: -30.92951] [Time:0.03728]
	[Generator][Epochs 48/50] [G epoch loss: -30.90

	[Generator][Epochs 24/50] [G epoch loss: -30.92438] [Time:0.02560]
	[Generator][Epochs 25/50] [G epoch loss: -30.83792] [Time:0.02558]
	[Generator][Epochs 26/50] [G epoch loss: -30.83037] [Time:0.02585]
	[Generator][Epochs 27/50] [G epoch loss: -30.74713] [Time:0.02580]
	[Generator][Epochs 28/50] [G epoch loss: -30.87081] [Time:0.02558]
	[Generator][Epochs 29/50] [G epoch loss: -30.78733] [Time:0.02574]
	[Generator][Epochs 30/50] [G epoch loss: -30.76998] [Time:0.02531]
	[Generator][Epochs 31/50] [G epoch loss: -30.84204] [Time:0.02578]
	[Generator][Epochs 32/50] [G epoch loss: -30.87840] [Time:0.02591]
	[Generator][Epochs 33/50] [G epoch loss: -30.84374] [Time:0.02594]
	[Generator][Epochs 34/50] [G epoch loss: -30.82148] [Time:0.02552]
	[Generator][Epochs 35/50] [G epoch loss: -30.82492] [Time:0.02615]
	[Generator][Epochs 36/50] [G epoch loss: -30.63301] [Time:0.02582]
	[Generator][Epochs 37/50] [G epoch loss: -30.71022] [Time:0.02556]
	[Generator][Epochs 38/50] [G epoch loss: -30.80